## Function to read json rows into a dataframe
json_df = pd.DataFrame.from_records(df['edp_raw_data_map'].apply(json.loads))

In [ ]:
import  plotly.express as px
import pandas as pd
import pyodbc
import json
from datetime import datetime 

def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 58

# Here is a function I use to read a file with a sql query
def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

# Then I use this to run the query and get the results as a dataframe
def run_query(query, dsn='edp-workbench-cshub'):
    cnxn = pyodbc.connect('DSN='+dsn, autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df


In [ ]:
df_27 = run_query(
     """SELECT
        CAST(eh.expert_id AS BIGINT) as "Employee",
        eh.expert_id as expert_id_string,
        eh.tenure_group as "TenureGroup",
        *
    FROM
        hive.care.l3_asurion_whole_home_expert_hierarchy eh
    WHERE 1=1
        AND eh.expert_id in (
            '293932','321930','364717','393198','426097',
            '536491','538855','547381','547655','548026',
            '548646','549667','550446','552121','552402',
            '554243','554487','561761','567519','567620',
            '568127','568568','568651','569153','569375',
            '569498','570227','572247','572815','572909',
            '573190','573192','573276','573573','573585',
            '574166','575731','575996','576565','576601',
            '576911','577073','577246','578299','579162',
            '579630','580895','580911','581015','581139',
            '581145','581275','581396','581694')
            AND CAST('2023-09-27' as DATE)
            BETWEEN eh.eff_start_dt and eh.eff_end_dt
""")

df_28 = run_query(
     """SELECT
        CAST(eh.expert_id AS BIGINT) as "Employee",
        eh.expert_id as expert_id_string,
        eh.tenure_group as "TenureGroup",
        *
    FROM
        hive.care.l3_asurion_whole_home_expert_hierarchy eh
    WHERE 1=1
        AND eh.expert_id in (
            '293932','321930','364717','393198','426097',
            '536491','538855','547381','547655','548026',
            '548646','549667','550446','552121','552402',
            '554243','554487','561761','567519','567620',
            '568127','568568','568651','569153','569375',
            '569498','570227','572247','572815','572909',
            '573190','573192','573276','573573','573585',
            '574166','575731','575996','576565','576601',
            '576911','577073','577246','578299','579162',
            '579630','580895','580911','581015','581139',
            '581145','581275','581396','581694')
            AND CAST('2023-09-28' as DATE)
            BETWEEN eh.eff_start_dt and eh.eff_end_dt
""")

In [ ]:
df_27[~df_27['Employee'].isin(df_28['Employee'])]

In [16]:
df =run_query(
"""
SELECT 
    element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') as "Employee",
    COALESCE(COUNT(element_at(VHE.edp_raw_data_map, 'Identities_messageSid')),0) as Messages,
    COALESCE(COUNT(DISTINCT element_at(VHE.edp_raw_data_map, 'Identities_SessionId')),0) as Sessions
FROM 
    hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
    AND edp_updated_date >= '2023-09-24'
GROUP BY
    element_at(VHE.edp_raw_data_map, 'ExtraData_agentId')"""
    )

display(df[df.columns[1:]].sum())
display(df.sort_values('Messages',ascending=False))


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_22264\1858826741.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


Messages    1604
Sessions     539
dtype: int64

,Employee,Messages,Sessions
19,hardCodedAgentId0,618,189
25,581015,92,31
23,581275,89,5
14,576911,88,22
3,581139,79,31
9,580911,75,28
6,569375,68,30
17,573192,58,23
1,576565,55,19
21,573190,54,22


In [6]:
display(df.sort_values('Messages'))
display(df.sum())

,Employee,Messages
24,516617,1
3,373685,3
7,573573,3
11,496312,4
22,oliver.wesche,7
5,364717,7
20,547655,8
2,572815,16
17,580895,24
14,573276,26


Employee    5693755769115728153736855735853647175759965735...
Messages                                                 1604
dtype: object

In [ ]:
# test_df = run_query(query_file_to_string('SQL/ExpertAssist.SQL'))
test_df.head()
pivot = pd.pivot_table(test_df, index='Date', values=['TotalUses','BackendLatency','CallsWithUse'], aggfunc=sum)
pivot['AvgLatency']= pivot['BackendLatency']/pivot['TotalUses']
px.line(pivot)

In [ ]:
for query in [
        'Sales.SQL',
        'HelixSearch.SQL',
        'EfficiencyMetrics.SQL',
        'CXP.SQL'
        ]:
    display(query+" "+timestamp())
    display(run_query(query_file_to_string('SQL/'+query)).head())

In [ ]:
run_query("SELECT * FROM hive.information_schema.tables where table_name like '%home%' and table_schema = 'care'")


In [ ]:
duplicateChecker = run_query(
"""
SELECT element_at(VHE.edp_raw_data_map, 'Identities_SessionId'), vsc.support_session_id
FROM
    hive.care.l1_verizon_home_events vhe
    LEFT OUTER JOIN
    hive.care.l3_verizon_soluto_crm vsc
    ON  element_at(VHE.edp_raw_data_map, 'Identities_SessionId') = vsc.support_session_id
WHERE 
    element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science' and
    element_at(VHE.edp_raw_data_map, 'Identities_messageSid') is not null and
    element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
""")

In [ ]:
display(duplicateChecker.columns)
duplicateChecker

In [ ]:
run_query(
"""
SELECT
    element_at(VHE.edp_raw_data_map,'Identities_SessionId'),
    substring(element_at(VHE.edp_raw_data_map, 'ExtraData_endRequestTime'),-10) as "Date1",
    substring(element_at(VHE.edp_raw_data_map, 'ExtraData_endRequestTime'),1, 10) as "Date2",
    substring(element_at(VHE.edp_raw_data_map, 'ExtraData_endRequestTime'),+10) as "Date3"
FROM
     hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
""")